## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-05-51-27 +0000,nyt,"Thaksin Shinawatra, Former Thai Premier, Clear...",https://www.nytimes.com/2025/08/21/world/asia/...
1,2025-08-22-05-44-58 +0000,bbc,Israel says it will start talks to free all ho...,https://www.bbc.com/news/articles/c754kknw2g2o...
2,2025-08-22-05-38-37 +0000,nyt,Pickett Fire Forces Evacuations in Napa County,https://www.nytimes.com/2025/08/21/us/pickett-...
3,2025-08-22-05-26-58 +0000,bbc,Beijing opposes 'bully' US for 50% tariffs on ...,https://www.bbc.com/news/articles/cger04e8g0vo...
4,2025-08-22-05-19-48 +0000,latimes,Video caught brutal assault at Rüfüs Du Sol sh...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
124,trump,53
95,new,21
42,ukraine,15
40,arrested,14
77,menendez,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
101,2025-08-21-22-35-03 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...,113
155,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,110
137,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,109
14,2025-08-22-03-52-00 +0000,wsj,A federal judge has ordered that the governmen...,https://www.wsj.com/politics/policy/federal-ju...,104
103,2025-08-21-22-28-59 +0000,bbc,Appeals court throws out Trump's $500m civil f...,https://www.bbc.com/news/articles/c5y09q1zgg8o...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
101,113,2025-08-21-22-35-03 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
84,68,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
9,64,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
1,59,2025-08-22-05-44-58 +0000,bbc,Israel says it will start talks to free all ho...,https://www.bbc.com/news/articles/c754kknw2g2o...
288,56,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
203,51,2025-08-21-17-07-24 +0000,nypost,North Korea’s top officers abruptly pulled out...,https://nypost.com/2025/08/21/world-news/north...
14,46,2025-08-22-03-52-00 +0000,wsj,A federal judge has ordered that the governmen...,https://www.wsj.com/politics/policy/federal-ju...
6,42,2025-08-22-04-56-10 +0000,nypost,Brute suspected of savagely knocking out woman...,https://nypost.com/2025/08/22/us-news/californ...
189,41,2025-08-21-17-41-08 +0000,bbc,"Record 111,000 UK asylum applications in past ...",https://www.bbc.com/news/articles/cwy1kxv8xewo...
116,37,2025-08-21-22-03-00 +0000,wsj,Two Fed officials offered competing assessment...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
